In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import math
import librosa
import tensorflow as tf
from collections import Counter

In [2]:
def load_data(dataset_path):
    # open file
    with open(dataset_path, "r") as fp:
        data = json.load(fp)

    # convert lists in to numpy arrays
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    return X, y

In [3]:
def prepare_datasets(test_size, valid_size):
    X, y = load_data("processed.json")

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=valid_size)
    
    return X_train, X_valid, X_test, y_train, y_valid, y_test

In [4]:
X_train, X_valid, X_test, y_train, y_valid, y_test = prepare_datasets(0.25, 0.2)

In [5]:
input_shape = (X_train.shape[1], X_train.shape[2])
model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        
        keras.layers.LSTM(units=64, return_sequences=True),
        keras.layers.LSTM(units=64),
        keras.layers.Dense(units=64, activation="relu"),
        keras.layers.Dropout(0.3),
        
        keras.layers.Dense(units=10, activation="linear")
    ]
)
optimizer = keras.optimizers.Adam(0.0001)
model.compile(
    optimizer=optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [6]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=30)

Epoch 1/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - accuracy: 0.1370 - loss: 2.2733 - val_accuracy: 0.3144 - val_loss: 2.0577
Epoch 2/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.3099 - loss: 1.9894 - val_accuracy: 0.3899 - val_loss: 1.7976
Epoch 3/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 13s 70ms/step - accuracy: 0.3652 - loss: 1.7638 - val_accuracy: 0.4433 - val_loss: 1.6250
Epoch 4/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 15s 78ms/step - accuracy: 0.4283 - loss: 1.6141 - val_accuracy: 0.4646 - val_loss: 1.5225
Epoch 5/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 10s 53ms/step - accuracy: 0.4710 - loss: 1.5229 - val_accuracy: 0.4927 - val_loss: 1.4483
Epoch 6/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.4852 - loss: 1.4723 - val_accuracy: 0.5174 - val_loss: 1.3981
Epoch 7/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.5039 - loss: 1.4053 - val_accuracy: 0.5214 - val_loss: 1.3614
Epoch 8/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.5379 - loss: 1.3447 - val_

In [7]:
test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Accuracy: {test_accuracy}, Error: {test_error}")

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6186 - loss: 1.1379
Accuracy: 0.6227473020553589, Error: 1.1224013566970825


In [8]:
model.save("weights/lstm_weights.keras")

In [9]:
# Function to extract MFCCs from audio file
def extract_mfccs_from_audio(
    file_path,
    segment_duration=3,
    n_mfcc=13,
    n_fft=2048,
    hop_length=512,
    sample_rate=22050,
):
    signal, sr = librosa.load(file_path, sr=sample_rate)

    # Calculate the number of samples per segment
    samples_per_segment = sample_rate * segment_duration
    expected_vector_length = math.ceil(samples_per_segment / hop_length)

    mfccs = []
    num_segments = int(len(signal) / samples_per_segment)

    for s in range(num_segments):
        start_sample = samples_per_segment * s
        finish_sample = start_sample + samples_per_segment

        if finish_sample > len(signal):
            break

        mfcc = librosa.feature.mfcc(
            y=signal[start_sample:finish_sample],
            sr=sr,
            n_fft=n_fft,
            n_mfcc=n_mfcc,
            hop_length=hop_length,
        )
        mfcc = mfcc.T

        if len(mfcc) == expected_vector_length:
            mfccs.append(mfcc.tolist())

    return np.array(mfccs)

In [10]:
# Load the weights
model.load_weights("weights/lstm_weights.keras")

In [11]:
mapping = [
    "blues",
    "classical",
    "country",
    "disco",
    "hiphop",
    "jazz",
    "metal",
    "pop",
    "reggae",
    "rock",
]

In [12]:
mfccs = extract_mfccs_from_audio("sample songs/master.mp3")
predictions = model.predict(mfccs)

probabilities = tf.nn.softmax(predictions, axis=-1)

predicted_classes = np.argmax(probabilities, axis=1)

class_counts = Counter(predicted_classes)
most_common_class = mapping[class_counts.most_common(1)[0][0]]

print(f"The predicted class for the song is: {most_common_class}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
The predicted class for the song is: metal
